In [42]:
# Import Modules
from bs4 import BeautifulSoup as bs
import requests 
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [43]:
# Setting up splinter
executable_path = {'executable_path': "/usr/local/bin/chromedriver"}
browser = Browser('chrome', **executable_path, headless = False)

In [47]:
# Setting url
mars_url = 'https://mars.nasa.gov/news/'

# Visit mars_url 
browser.visit(mars_url)

# Scrape page into soup
html = browser.html
mars_soup = bs(html, 'html.parser')

In [48]:
# Getting the news title
news_title = mars_soup.find_all('div', class_='content_title')[0].text

# Getting the description 
news_p = mars_soup.find_all('div', class_='article_teaser_body')[0].text

In [59]:
#Setting url
jpg_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

#Visit jpg_url
browser.visit(jpg_url)

#Scraping the description 
html = browser.html
jpg_soup = bs(html, 'html.parser')

In [60]:
#Finding the url string
url_string = jpg_soup.find_all('img', class_='main_')[3]['src']
url_string

IndexError: list index out of range

In [61]:
#Adding both urls together to get full url
featured_image_url = jpg_url + url_string
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA24244-640x350.jpg'

In [11]:
#Findint the table url
table_url = 'https://space-facts.com/mars/'

In [12]:
#Bringing in the table
tables = pd.read_html(table_url)

#Displaying the table 
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [13]:
#Setting the table to type 2
mars_facts = tables[2]

#Setting column names
mars_facts.columns = ['Measurement','Value']

#Displaying table 
mars_facts

,Measurement,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [14]:
#Converting table to html
mars_html = mars_facts.to_html()

In [15]:
#Replace \n with an empty string 
mars_html = mars_html.replace('\n','')
print(mars_html)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Measurement</th>      <th>Value</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronome

In [16]:
#Setting the urls
base_url = 'https://astrogeology.usgs.gov'
photos_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

#Visiting photos_url
browser.visit(photos_url)

#Scraping the description
html = browser.html
photos_soup = bs(html, 'html.parser')

In [17]:
#Finding the titles and jpegs for all hemisphere of mars
all_hemispheres = photos_soup.find('div', class_='collapsible results')
hemisphere_titles = all_hemispheres.find_all('div', class_='item')

#Setting hemisphere_image_urls to a list
hemisphere_image_urls = []

#Setting hem_images_dict to a dictionary
hem_images_dict = {}

#Iterating through hemisphere_titles
for n in hemisphere_titles:
    
    #Finding the title for each hemisphere of mars 
    hem = n.find('div', class_='description')
    title = hem.h3.text
    
    #Fidning the url for the picture links 
    pic_url = hem.a['href']
    
    #Visit the large pics of every hemisphere of mars
    browser.visit(base_url+pic_url)
    
    #Scraping the description 
    pic_html = browser.html
    pic_soup = bs(pic_html, 'html.parser')
    
    #Finding the url for large pics
    mars_pic_link = pic_soup.find('div', class_='downloads')
    image_url = mars_pic_link.find('li').a['href']
    
    #Setting name of the dictionary keys 
    hem_images_dict['title']=title
    hem_images_dict['image_url']=image_url
    
    #Appending each dictionary entry to a list 
    hemisphere_image_urls.append(hem_images_dict)

#Printing the url
print(hemisphere_image_urls)

[{'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [18]:
#Setting all entries into a dictionary 
mars_dict = {'news_title':news_title,
            'news_p':news_p,
            'featured_image_url':featured_image_url,
            'hemisphere_image_urls':hemisphere_image_urls,
            'mars_facts':mars_html
            }